# Image Classification experiment for CIFAR-10 dataset 

CIFAR-10 (https://www.cs.toronto.edu/~kriz/cifar.html) contains 60,000 32*32 color images in 10 classes, with 6000 images per class. 

Task is to build a multiclass image classifier using 2D Convolutional Neural Networks (2D CNNs) to achieve the best possible classification accuracy for the test set.   

In [4]:
#Importing required python packages
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torchnet.meter.confusionmeter as cm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import utils

import models.resnet as resnet
import models.simple_net as simple_net
import models.simple_net_bn as simple_net_bn

In [5]:
# Assign CUDA device for computations:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Data Transformation 

In [6]:
#transformation for training image set - with flip augmentation
transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

#transformation for testing image set
transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])


## Load Dataset

In [7]:
#Define the mini-batch size
batch_size = 128

#datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)

#dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Dataset Analysis

In [8]:
#Find the dataset sizes and number of batches

#train set size
train_set_size = len(trainset)

#test set size
test_set_size = len(testset)

#Number of mini-batches in trainset /testset
train_batches = len(trainloader)
test_batches = len(testloader)

print("Train set size : " + str(train_set_size))
print("Training batches : " + str(train_batches))
print("Test set size : " + str(test_set_size))
print("Testing batches : " + str(test_batches))

Train set size : 50000
Training batches : 391
Test set size : 10000
Testing batches : 79


## Data Visualization

In [9]:
#Visualize 4 random training images
viewloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# get some random training images
dataiter = iter(viewloader)
images, labels = next(dataiter)


# show images
utils.imshow(torchvision.utils.make_grid(images))
utils.imshowtransform(torchvision.utils.make_grid(images))

# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'

## Initializing Networks

In [ ]:
simplenet = simple_net.SimpleNet()
simplenet_bn = simple_net_bn.SimpleNet_bn()
resnet34 = resnet.ResNet34()

# pushing the network to the GPU
net = simplenet_bn.to(device)

print(net)

## Training the ANN

In [ ]:
#Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

#Define the optimizer and the learning rate
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
#lists for generating graphs
epochs = list()
training_loss = list()

# loop over the dataset multiple times
for epoch in range(50):  

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    
        # print statistics
        running_loss += loss.item()
    print('[Epoch : %d] train_loss: %.3f' %
          (epoch + 1, running_loss / train_batches))
    epochs.append(epoch)
    training_loss.append(running_loss/train_batches)
    running_loss = 0.0
    scheduler.step()
    
print('Finished Training')

In [ ]:
#Plot the epoch vs. training loss
plt.figure(1)
plt.title("Training Loss - CIFAR-10 Classification")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(epochs, training_loss, color='r', label="Training Loss")
plt.legend()
plt.show()

## Loading a trained ResNet34 model

In [ ]:
#Saving trained model
#torch.save(net.state_dict(), "trained_resnet_model.pth")

#Loading the model
model = resnet.ResNet34().to(device)
model.load_state_dict(torch.load("trained_resnet_model.pth"))

## Test model

In [ ]:
#Testing the trained model with testing data
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
#Evaluate the class-wise accuracy
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
#Get the confusion matrix for testing data
confusion_matrix = cm.ConfusionMeter(10)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        confusion_matrix.add(predicted, labels)
    #print(confusion_matrix.conf)

#Confusion matrix as a heatmap
con_m = confusion_matrix.conf
df_con_m = pd.DataFrame(con_m, index= [i for i in classes], columns = [i for i in classes])
sn.set(font_scale= 1.1)
sn.heatmap(df_con_m, annot=True,fmt='g' ,  annot_kws={"size" : 10}, cbar = False, cmap="Blues") 